In [1]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

In [15]:
#Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 50
batchsize = 20
batchsize_test = 20
learning_rate = 0.01
momentum = 0.5
log_interval = 60
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
TEST_DATA_PATH = "../Data/CATS_DOGS/test"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True,  num_workers=4)

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batchsize_test, shuffle=True, num_workers=4) 



In [3]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 20, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(20, 32, kernel_size=3)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = torch.nn.Linear((64*17*17), 50)
        self.fc2 = torch.nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, (64*17*17))
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [16]:
#Network model
network = Network()
network.to(device)

#Stochastic gradient decent
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

criterion = torch.nn.BCELoss()

In [5]:
summary(network, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 20, 148, 148]             560
            Conv2d-2           [-1, 32, 72, 72]           5,792
            Conv2d-3           [-1, 64, 34, 34]          18,496
            Linear-4                   [-1, 50]         924,850
            Linear-5                    [-1, 1]              51
Total params: 949,749
Trainable params: 949,749
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 5.17
Params size (MB): 3.62
Estimated Total Size (MB): 9.05
----------------------------------------------------------------


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [19]:
##Just for testign
test = iter(train_data_loader)
img, target= next(test)

out = network(img.to(device))
#test_loss += F.binary_cross_entropy(output, target).item()
#plt.imshow(img[9][0])

#img.shape


tensor([0.5180], device='cuda:0', grad_fn=<SelectBackward0>)

In [17]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_data_loader.dataset) for i in range(num_epochs + 1)]

In [12]:
def test():
    #Sets the module in evaluation mode
    network.eval()
    test_loss = 0
    correct = 0
    #correct.to(device)
    
    #dont update dynamic computation graph
    with torch.no_grad():
        #for every example in test
        for data, target in test_data_loader:
            
            target = target.view(-1, 1).float()
            target.to(device)
            
            #evaluate the model
            output = network(data.to(device))
            
            #acumulate the loss
            test_loss += F.binary_cross_entropy(output, target.to(device)).item()
            
            pred_cls = output.round()
            correct += pred_cls.eq(target.view(-1, 1).to(device)).sum() 
        
    test_loss /= len(test_data_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_data_loader.dataset), 100. * correct / len(test_data_loader.dataset)))

In [43]:
##Just for testign
test_loss = 0
correct = 0
test = iter(test_data_loader)
img, target= next(test)

out = network(img.to(device))

pred = out.round()
t = pred.eq(target.view(-1, 1).to(device))
t

tensor([[False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True]], device='cuda:0')

In [18]:
#train

test()

for epoch in range(1, num_epochs+1):
    
    for batch_idx, (data, target) in enumerate(train_data_loader):
        
        network.train()
        
        #print(f'b {batch}')
        
        #forward pass
        out = network(data.to(device))
        
        #Use negative log likelihood loss.
        loss = criterion(out, target.view(-1, 1).float().to(device))
        
        
        #with this gradients are calculated
        loss.backward()
    
        #update gradients
        optimizer.step()
        
        #Set gradients to zero
        optimizer.zero_grad()

        #Display iteration statistics
        if batch_idx % log_interval == 0:
            
            #print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_data_loader.dataset),100. * batch_idx / len(train_data_loader), loss.item()))
        
    test()
    
    


Test set: Avg. loss: 0.0347, Accuracy: 3125/6251 (50%)

Train Epoch: 1 [0/18743 (0%)]	Loss: 0.694746
Train Epoch: 1 [1200/18743 (6%)]	Loss: 0.687647
Train Epoch: 1 [2400/18743 (13%)]	Loss: 0.699549
Train Epoch: 1 [3600/18743 (19%)]	Loss: 0.681731
Train Epoch: 1 [4800/18743 (26%)]	Loss: 0.650089
Train Epoch: 1 [6000/18743 (32%)]	Loss: 0.673029
Train Epoch: 1 [7200/18743 (38%)]	Loss: 0.633342
Train Epoch: 1 [8400/18743 (45%)]	Loss: 0.645627
Train Epoch: 1 [9600/18743 (51%)]	Loss: 0.657408
Train Epoch: 1 [10800/18743 (58%)]	Loss: 0.678526


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [12000/18743 (64%)]	Loss: 0.561763
Train Epoch: 1 [13200/18743 (70%)]	Loss: 0.596372
Train Epoch: 1 [14400/18743 (77%)]	Loss: 0.559260
Train Epoch: 1 [15600/18743 (83%)]	Loss: 0.687345
Train Epoch: 1 [16800/18743 (90%)]	Loss: 0.712594
Train Epoch: 1 [18000/18743 (96%)]	Loss: 0.718519

Test set: Avg. loss: 0.0315, Accuracy: 3982/6251 (64%)

Train Epoch: 2 [0/18743 (0%)]	Loss: 0.576999
Train Epoch: 2 [1200/18743 (6%)]	Loss: 0.583160
Train Epoch: 2 [2400/18743 (13%)]	Loss: 0.486749
Train Epoch: 2 [3600/18743 (19%)]	Loss: 0.718111
Train Epoch: 2 [4800/18743 (26%)]	Loss: 0.520314
Train Epoch: 2 [6000/18743 (32%)]	Loss: 0.681272
Train Epoch: 2 [7200/18743 (38%)]	Loss: 0.503358
Train Epoch: 2 [8400/18743 (45%)]	Loss: 0.630494
Train Epoch: 2 [9600/18743 (51%)]	Loss: 0.540679
Train Epoch: 2 [10800/18743 (58%)]	Loss: 0.546653
Train Epoch: 2 [12000/18743 (64%)]	Loss: 0.590082
Train Epoch: 2 [13200/18743 (70%)]	Loss: 0.692892
Train Epoch: 2 [14400/18743 (77%)]	Loss: 0.729382


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [15600/18743 (83%)]	Loss: 0.647271
Train Epoch: 2 [16800/18743 (90%)]	Loss: 0.739437
Train Epoch: 2 [18000/18743 (96%)]	Loss: 0.707266

Test set: Avg. loss: 0.0268, Accuracy: 4568/6251 (73%)

Train Epoch: 3 [0/18743 (0%)]	Loss: 0.479781
Train Epoch: 3 [1200/18743 (6%)]	Loss: 0.795115
Train Epoch: 3 [2400/18743 (13%)]	Loss: 0.533618
Train Epoch: 3 [3600/18743 (19%)]	Loss: 0.466122
Train Epoch: 3 [4800/18743 (26%)]	Loss: 0.568130
Train Epoch: 3 [6000/18743 (32%)]	Loss: 0.473232
Train Epoch: 3 [7200/18743 (38%)]	Loss: 0.617662
Train Epoch: 3 [8400/18743 (45%)]	Loss: 0.543164
Train Epoch: 3 [9600/18743 (51%)]	Loss: 0.615983
Train Epoch: 3 [10800/18743 (58%)]	Loss: 0.525211
Train Epoch: 3 [12000/18743 (64%)]	Loss: 0.525978
Train Epoch: 3 [13200/18743 (70%)]	Loss: 0.548553


...

Train Epoch: 48 [10800/18743 (58%)]	Loss: 0.032796
Train Epoch: 48 [12000/18743 (64%)]	Loss: 0.002250
Train Epoch: 48 [13200/18743 (70%)]	Loss: 0.002670
Train Epoch: 48 [14400/18743 (77%)]	Loss: 0.002722
Train Epoch: 48 [15600/18743 (83%)]	Loss: 0.001749
Train Epoch: 48 [16800/18743 (90%)]	Loss: 0.034302
Train Epoch: 48 [18000/18743 (96%)]	Loss: 0.000030

Test set: Avg. loss: 0.0754, Accuracy: 5260/6251 (84%)

Train Epoch: 49 [0/18743 (0%)]	Loss: 0.004764
Train Epoch: 49 [1200/18743 (6%)]	Loss: 0.002427
Train Epoch: 49 [2400/18743 (13%)]	Loss: 0.027016
Train Epoch: 49 [3600/18743 (19%)]	Loss: 0.007365
Train Epoch: 49 [4800/18743 (26%)]	Loss: 0.029782
Train Epoch: 49 [6000/18743 (32%)]	Loss: 0.000296
Train Epoch: 49 [7200/18743 (38%)]	Loss: 0.101880
Train Epoch: 49 [8400/18743 (45%)]	Loss: 0.005101
Train Epoch: 49 [9600/18743 (51%)]	Loss: 0.010746
Train Epoch: 49 [10800/18743 (58%)]	Loss: 0.006501


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 49 [12000/18743 (64%)]	Loss: 0.127300
Train Epoch: 49 [13200/18743 (70%)]	Loss: 0.004581
Train Epoch: 49 [14400/18743 (77%)]	Loss: 0.015186
Train Epoch: 49 [15600/18743 (83%)]	Loss: 0.133934
Train Epoch: 49 [16800/18743 (90%)]	Loss: 0.019150
Train Epoch: 49 [18000/18743 (96%)]	Loss: 0.003828

Test set: Avg. loss: 0.0976, Accuracy: 5185/6251 (83%)

Train Epoch: 50 [0/18743 (0%)]	Loss: 0.004548
Train Epoch: 50 [1200/18743 (6%)]	Loss: 0.001827
Train Epoch: 50 [2400/18743 (13%)]	Loss: 0.001101
Train Epoch: 50 [3600/18743 (19%)]	Loss: 0.001422
Train Epoch: 50 [4800/18743 (26%)]	Loss: 0.027102
Train Epoch: 50 [6000/18743 (32%)]	Loss: 0.001770
Train Epoch: 50 [7200/18743 (38%)]	Loss: 0.065278
Train Epoch: 50 [8400/18743 (45%)]	Loss: 0.013950
Train Epoch: 50 [9600/18743 (51%)]	Loss: 0.028629


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 50 [10800/18743 (58%)]	Loss: 0.000202
Train Epoch: 50 [12000/18743 (64%)]	Loss: 0.019987
Train Epoch: 50 [13200/18743 (70%)]	Loss: 0.003414
Train Epoch: 50 [14400/18743 (77%)]	Loss: 0.000062
Train Epoch: 50 [15600/18743 (83%)]	Loss: 0.035409
Train Epoch: 50 [16800/18743 (90%)]	Loss: 0.000136
Train Epoch: 50 [18000/18743 (96%)]	Loss: 0.041978

Test set: Avg. loss: 0.0788, Accuracy: 5238/6251 (84%)



In [13]:
print("torch.cuda.memory_allocated: %fMB"%(torch.cuda.memory_allocated(0)/1024/1024))
print("torch.cuda.memory_reserved: %fMB"%(torch.cuda.memory_reserved(0)/1024/1024))
print("torch.cuda.max_memory_reserved: %fMB"%(torch.cuda.max_memory_reserved(0)/1024/1024))


torch.cuda.memory_allocated: 10.877441MB
torch.cuda.memory_reserved: 166.000000MB
torch.cuda.max_memory_reserved: 166.000000MB


### ToDo

- Create a section in which the network is described
- Check other kind of accuaracy measurements
- Check why having a 24 batch doesn't work
- Check how to maximize utilization of GPU, does this affect training results?
- Monitor gpu usage, memory, SM usage?